In [ ]:
import numpy as np
class Customer:
    def __init__(self, arrive_time, quantity, service_time, x, y):
        self.arrive_time = arrive_time
        self.quantity = quantity
        self.service_time = service_time
        self.x = x
        self.y = y


class Truck_Solution:
    def __init__(self, assigned_customers, recived_truck, recived_drone, arrive_time, leave_time):
        self.assigned_customers = assigned_customers
        self.recived_truck = recived_truck
        self.recived_drone = recived_drone
        self.arrive_time = arrive_time
        self.leave_time = leave_time

class Drone_Trip:
    def __init__(self, assigned_customers, recived_drone, arrive_time, leave_time):
        self.assigned_customers = assigned_customers
        self.recived_drone = recived_drone
        self.arrive_time = arrive_time
        self.leave_time = leave_time

class Drone_Solution:
    def __init__(self, num_of_trips, trip_list):
        self.num_of_trips = num_of_trips
        self.trip_list = trip_list

        
class Problem:
    def __init__(self, customer_list, number_of_trucks, number_of_drones, distance_matrix_truck,
                distance_matrix_drone, truck_capacity, drone_capacity, drone_energy,
                speed_of_truck, speed_of_drone, launch_time, land_time,
                energy_consumption_rate, weight_of_drone,
                cost_truck_unit, cost_drone_unit):
        self.customer_list = customer_list
        self.number_of_trucks = number_of_trucks
        self.number_of_drones = number_of_drones
        self.distance_matrix_truck = distance_matrix_truck
        self.distance_matrix_drone = distance_matrix_drone
        self.truck_capacity = truck_capacity
        self.drone_capacity = drone_capacity
        self.drone_energy = drone_energy
        self.speed_of_truck = speed_of_truck
        self.speed_of_drone = speed_of_drone
        self.launch_time = launch_time
        self.land_time = land_time
        self.energy_consumption_rate = energy_consumption_rate
        self.weight_of_drone = weight_of_drone
        self.cost_truck_unit = cost_truck_unit
        self.cost_drone_unit = cost_drone_unit

    def check_capacity_truck_contraint(self, truck_solution: Truck_Solution):
        customer_length = len(truck_solution.assigned_customers)
        if customer_length == 0:
            return True
        total_goods = 0
        for i in range(customer_length):
            total_goods = total_goods + truck_solution.recived_truck[i] + truck_solution.recived_drone[i]
        if total_goods > self.truck_capacity:
            return False
        return True
    
    def check_capacity_drone_trip_constraint(self, drone_trip: Drone_Trip):
        if len(drone_trip.assigned_customers) == 0:
            return True
        total_goods = 0
        for i in range(len(drone_trip.assigned_customers)):
            total_goods = total_goods + drone_trip.recived_drone[i]
        if total_goods > self.drone_capacity:
            return False
        return True
    
    def check_capacity_drone_constraint(self, drone_solution: Drone_Solution):
        for i in range(drone_solution.num_of_trips):
            if self.check_capacity_drone_trip_constraint(drone_solution.trip_list[i]) == False:
                return False
        return True
    

    def check_energy_drone(self, drone_trip: Drone_Trip):
        number_point = len(drone_trip.assigned_customers)
        if number_point == 0:
            return True
        total_energy = 0
        total_goods = 0
        for i in range(number_point - 1):
            u, u_next = drone_trip.assigned_customers[i], drone_trip.assigned_customers[i+1]
            total_energy = total_energy + self.energy_consumption_rate*(self.weight_of_drone + total_goods)*(self.distance_matrix_drone[u][u_next]/self.speed_of_drone + self.launch_time)
            if drone_trip.recived_drone[i] != 0:
                wait_time = max(self.customer_list[u_next].arrive_time - drone_trip.arrive_time[i], 0)
                total_energy = total_energy + self.energy_consumption_rate*(self.weight_of_drone + total_goods)*(self.land_time + wait_time)
                total_goods = total_goods + drone_trip.recived_drone[i]
                total_energy = total_energy + self.energy_consumption_rate*(self.weight_of_drone + total_goods)*self.customer_list[u_next].service_time
            else:
                wait_time = max(0, drone_trip.leave_time[i+1] - drone_trip.arrive_time[i+1])
                total_energy = total_energy + self.energy_consumption_rate*(self.weight_of_drone + total_goods)*wait_time
        if total_energy > self.drone_energy:
            return False
        else:
            return True
        
    def cal_truck_cost(self, truck_solution: Truck_Solution):
        if len(truck_solution.assigned_customers) == 0:
            return 0
        else:
            total_cost = self.distance_matrix_truck[0][truck_solution.assigned_customers[0]]*self.cost_truck_unit
            for i in range(len(truck_solution.assigned_customers)-1):
                total_cost = total_cost + self.distance_matrix_truck[truck_solution.assigned_customers[i]][truck_solution.assigned_customers[i+1]]*self.cost_truck_unit
            return total_cost
        
    def cal_drone_trip_cost(self, drone_trip: Drone_Trip):
        num_point = len(drone_trip)
        if num_point == 0:
            return 0
        else:
            total_cost = 0
            for i in range(num_point - 1):
                total_cost = total_cost + self.distance_matrix_drone[drone_trip.assigned_customers[i]][drone_trip.assigned_customers[i+1]]*self.cost_drone_unit
            return total_cost
        
    def cal_drone_cost(self, drone_solution: Drone_Solution):
        total_cost = 0
        if drone_solution.num_of_trips == 0:
            return 0
        for trip in drone_solution.trip_list:
            total_cost = total_cost + self.cal_drone_trip_cost(trip)
        return total_cost
    
    def cal_total_cost(self, truck_solution_list, drone_solution_list):
        total_cost = 0
        for truck_solution in truck_solution_list:
            total_cost = total_cost + self.cal_truck_cost(truck_solution)
        for drone_solution in drone_solution_list:
            total_cost = total_cost + self.cal_drone_cost(drone_solution)
        return total_cost
    

    def customer_wait_max(self, truck_solution_list, drone_solution_list):
        customer_wait_time = np.zeros(len(self.customer_list))
        for truck_solution in truck_solution_list:
            num_point = len(truck_solution.assigned_customers)
            if num_point == 0:
                continue
            else:
                for i in range(num_point):
                    if truck_solution.recived_truck[i] == 0:
                        continue
                    else:
                        customer = truck_solution.assigned_customers[i]
                        customer_wait_time[customer] = max(truck_solution.arrive_time[i] - self.customer_list[customer].arrive_time, 0)
        
        for drone_solution in drone_solution_list:
            for drone_trip in drone_solution:
                num_point = len(drone_trip.assigned_customers)
                if num_point == 0:
                    continue
                else:
                    for i in range(num_point):
                        if drone_trip.recived_drone[i] == 0:
                            continue
                        else:
                            customer = drone_trip.assigned_customers[i]
                            customer_wait_time[customer] = max(drone_trip.arrive_time[i] - self.customer_list[customer].arrive_time, 0)
        return np.max(customer_wait_time)
    
    def cal_distance_truck(self, truck_solution: Truck_Solution):
        total_distance = 0
        num_point = len(truck_solution)
        if num_point == 0:
            return total_distance
        else:
            total_distance = total_distance + self.distance_matrix_truck[0][truck_solution.assigned_customers[0]] + self.distance_matrix_truck[truck_solution.assigned_customers[-1]][0]
            for i in range(num_point):
                u, u_next = truck_solution.assigned_customers[i], truck_solution.assigned_customers[i+1]
                total_distance = total_distance + self.distance_matrix_truck[u][u_next]
            return total_distance
        
    def cal_truck_fairness(self, truck_soltuon_list):
        num_truck = len(truck_soltuon_list)
        distance_list = np.zeros(num_truck)
        for i in range(num_truck):
            distance_list = self.cal_distance_truck(truck_soltuon_list[i])
        return np.std(distance_list)


In [22]:
import sys
import os
src_path = r"./"
sys.path.append(src_path)

In [ ]:
class State:
    def __init__(self, problem):
        self.truck = []             # location of trucks
        self.drones = []            # location of drones
        self.time = 0

SyntaxError: incomplete input (655263051.py, line 1)

In [ ]:
from population import Individual

'''
[1, -1, 3, 7, 4, 0, 5, 6, -1, 8, 2]
[0,  0, 0, 1, 0, 0, 1, 0,  0, 1, 0]

0: truck
1: drone

0: truck split
-1: drone split
1 '0' means there are 2 trucks
2 '-1' means there are 3 drones

truck route: [1, 3, 4], [6, 2]
drone route: [], [7, 5], [8]

'''
def extract_routes(chromosome):
    truck_routes = []
    drone_routes = []
    tmp_truck_route = []
    tmp_drone_route = []
    
    for i in range(len(chromosome[0])):
        if chromosome[0][i] == 0:
            truck_routes.append(tmp_truck_route)
            tmp_truck_route = []
            continue
        elif chromosome[0][i] == -1:
            drone_routes.append(tmp_drone_route)
            tmp_drone_route = []
            continue
        
        if chromosome[1][i] == 0:
            tmp_truck_route.append(chromosome[0][i])
        elif chromosome[1][i] == 1:
            tmp_drone_route.append(chromosome[0][i])
    
    truck_routes.append(tmp_truck_route)
    drone_routes.append(tmp_drone_route)
    print("Truck routes:", truck_routes)
    print("Drone routes:", drone_routes)
    
    return truck_routes, drone_routes


def decode(individual: Individual, problem: Problem):
    chromosome = individual.chromosome
    truck_routes, drone_routes = extract_routes(chromosome)
    
    # sort delivery sequence by arrival time
    for route in truck_routes:
        route.sort(key=lambda cust: problem.customer_list[cust].arrive_time)
    
    for route in drone_routes:
        route.sort(key=lambda cust: problem.customer_list[cust].arrive_time)
        
    if problem is None:
        return
    
    timestamps = [problem.customer_list[cust].arrive_time for cust in problem.customer_list]
    timestamps.sort()    
    
    for t in timestamps:
        update_state()        
    # visited = [False for _ in range(len(problem.customer_list)+1)]
    # visited[0] = True
    # priority_queue = []
    
    # this loop will terminate when all customers are visited and all used drones are collected or return to depot
    while True:
        # select the lauching drone:
        #   choose the drone which has the node with lowest arrival time
        
        #   
        # select truck node the selected drone will launch from
        #   choose the closest one to the first node the drone will visit in its route
        #   + if the node is still infeasible, select the one with the second lowest arrival time
        
        # pseudocode:
        #   + iterating through the list of drone routes which is ordered by the first unvisited customer's arrival time
        #       ? how many customers should be picked up by the drone in this trip
        #       + iterating through the list of unvisited customers in the current drone route
        #           ? should be picked up by the drone in this trip
        #   + update the state of truck routes
        #   
        
        pass
    
            

In [26]:
chromosome = [[1, -1, 3, 7, 4, 0, 5, 6, -1, 8, 2],
              [0,  0, 0, 1, 0, 0, 1, 0,  0, 1, 0]]

indi = Individual(chromosome)
decode(indi, None)

Truck routes: [[1, 3, 4], [6, 2]]
Drone routes: [[], [7, 5], [8]]
